In [22]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Dropout

### 데이터 준비하기
 - https://tykimos.github.com/Keras/warehouse/2017-3-8_CNN_Getting_Started_handwriting_shape.zip

### 데이터셋 생성하기

In [23]:
# 랜덤시드 고정시키기
np.random.seed(3)

train_datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=10,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 shear_range=0.7,
                                 zoom_range=[0.9, 2.2],
                                 horizontal_flip=True,
                                 vertical_flip=True,
                                 fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    'data/hard_handwriting_shape/train',
    target_size = (24, 24),
    batch_size = 3,
    class_mode = 'categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'data/hard_handwriting_shape/test',
    target_size = (24, 24),
    batch_size = 3,
    class_mode = 'categorical'
)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


### 모델 구성하기
ImportError: Could not import PIL.Image. The use of `array_to_img` requires PIL  
이와 같은 에러 발생 시, 아래와 같이 조치함  
conda remove pillow  
pip install pillow  

In [24]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),
                activation='relu',
                input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               819328    
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
_________________________________________________________________


### 모델 학습과정 설정하기

In [25]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 모델 학습시키기

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=15*100,
    epochs=200,
    validation_data=test_generator,
    validation_steps=5
)

Epoch 1/200
1500/1500 [==============================] - 9s 6ms/step - loss: 0.6619 - acc: 0.6991 - val_loss: 0.1489 - val_acc: 0.9333
Epoch 2/200
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3185 - acc: 0.8840 - val_loss: 0.0267 - val_acc: 1.0000
Epoch 3/200
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2172 - acc: 0.9216 - val_loss: 0.6051 - val_acc: 0.8667
Epoch 4/200
1500/1500 [==============================] - 9s 6ms/step - loss: 0.1719 - acc: 0.9398 - val_loss: 0.1057 - val_acc: 0.9333
Epoch 5/200
1500/1500 [==============================] - 9s 6ms/step - loss: 0.1354 - acc: 0.9533 - val_loss: 1.0652 - val_acc: 0.8000
Epoch 6/200
1500/1500 [==============================] - 9s 6ms/step - loss: 0.1156 - acc: 0.9591 - val_loss: 0.0334 - val_acc: 1.0000
Epoch 7/200
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0888 - acc: 0.9669 - val_loss: 0.0353 - val_acc: 1.0000
Epoch 8/200
1500/1500 [==============================] 

### 모델 평가하기

In [12]:
print ("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
acc: 66.67%


### 모델 사용하기

In [13]:
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
{'circle': 0, 'rectangle': 1, 'triangle': 2}
[[1.000 0.000 0.000]
 [1.000 0.000 0.000]
 [0.000 1.000 0.000]
 [0.000 1.000 0.000]
 [1.000 0.000 0.000]
 [0.736 0.222 0.042]
 [1.000 0.000 0.000]
 [0.004 0.000 0.996]
 [0.000 1.000 0.000]
 [0.736 0.222 0.042]
 [1.000 0.000 0.000]
 [0.000 0.000 1.000]
 [1.000 0.000 0.000]
 [0.089 0.013 0.898]
 [0.004 0.593 0.403]]
